In [1]:
using Revise
using VariPEPS
using TensorKit
using OptimKit
#using JLD2 #needed?

In this notebook we show the very basic functionality of the workhorse funtions of the `VariPEPS.jl` code

Specify the Bond-dimension `d`, the environment-bond-dimension `χ`. 

In [2]:
χ = 8
d = 2

2

**Here will follow some info on coarse graining and so forth...**

Here specify the unit cell. For this we define an array "Pattern_arr" that represents the unit cell structure.\
The structure is [x-coodinate, y-coordinate].\
If two positions within the Pattern_arr are filled with the same number, we assume the tensors at these positions to be identical.\
The choice below corresponds to a unit cell like:   
A   B \
B   A

In [3]:
Pattern_arr = Array{Any}(undef,1,1)

Pattern_arr[1,1] = 1;

For our first example we are going to look at the Heisenberg-antiferromagnet on the square lattice.\
Here we specify the keywords corresponding to this choice of model and lattice on in a tuple.

Regarding the keywords:\
**lattice**: tells us which lattice we use.\
**model**: which Hamiltonian are we using.\
**Ham_parameters**: Give values for the parameters in the Hamiltonian. Here we choose an anti-ferromagnetic model (J=1) with no magnetic field term (h = 0) which points in no direction (dir = [0,0,0])\
**Projector_type**: During the CTMRG algorithm we need projectors. These can be choosen in different ways. Here we choose the :half option. An other more expensive choice would be :fullfishman.\
**Space_type**: This tells us what the basic vector space is that we want to work with. As we have a TRS model here we can choose a real wavefunction so we choose ℝ. Otherwise choose ℂ.\

In [4]:
keywords = (lattice = :kagome, model = :Heisenberg_kagome, Ham_parameters = (J = 1, h = 0 , dir = [0,0,0]), Projector_type = :half, Space_type = ℂ)

(lattice = :kagome, model = :Heisenberg_kagome, Ham_parameters = (J = 1, h = 0, dir = [0, 0, 0]), Projector_type = :half, Space_type = ℂ)

The functiion 
```
initialize_PEPS()
```
initializes the PEPS-tensors. It takes 3 basic inputs:
1. The first input is the Bond dimension, 
2. the second input is the dimension of the local Hilbert space, --> This is 8 in the current example, as we will be coarse graining 3 spin 1/2 sites.
3. the third one specified how many tensors need to be generated. --> In this example 2 since we have two inequivalent tensors in the unit cell.
The additional keywords can be used to specify some addtional options.

What is created here is just an array of TensorMaps with the right number of legs with the desired dimensionality.

Note that we can also enforce local symmetries and only optimize on the parametes for the local tensors that have this symmetry. --> Add mini example later.

In [5]:
loc_in = initialize_PEPS(d, 8, 1; seed = 1235, lattice = :kagome, Number_type = ComplexF64, Space_type = ℂ); 

The ctmrg function performs a CTM-RG algorithm. This means it calculates environment tensors and that uses those to calculate the energy density.
If we put the option 
```
conv_info = true
```
The function shows some output that idicates the status of the algorithm. 

The function takes as input the objects we created above: A set of local PEPS tensors (loc_in) as well as some information on the unit cell (Pattern_arr).
It can also be used to evaluate some list of observables. For the Heisenberg model these are by default the spin matrices. However the user can also specify a list of alternative observables to calculate. *this works analogously to specifying your own model and is explained in a seperate notebook*. Note that if we want to get the expectation values of the spin matrices we need to choose Space_type = ℂ above as they are complex matrices.


In [6]:
ctmrg_res = ctmrg(loc_in, χ, Pattern_arr; keywords..., observ = true, conv_info = false)

(0.05834679475447388, Any[-0.025558645107357676 - 2.6631701280130283e-17im, -0.025558645107357676 - 2.6631701280130283e-17im, -0.025558645107357676 - 2.6631701280130283e-17im, 0.0029560718762820333 - 5.1422803494370277e-17im, -0.0452451732549729 - 2.968822307985751e-17im, 0.017616656843871207 + 3.972505566638601e-17im, -0.03679391519317054 + 5.317184308108422e-18im, -0.01313419180070575 + 5.063917073667882e-17im, -0.08234560956260958 + 1.52016254219393e-17im])

Some comments on additional options (skip on first reading): 

1. With the option svd_type = :GKL one can enable the iterative calculation of the singular values in the CTMRG algorithm.
This can be substantially faster for larger bond dimensions.
Note that in some cases, the GKL algorithm, that can be used for the SVD in the calculation of the projectors finds invariant subspaces and fails. 
In these cases the code will fall back to conventional SVD algorithms.

2. In the "energy_and_gradient" function as well as in the "ctmrg" function we have a keyword "adjust_χ" which is by default set to false. In that case the 
function works normally. However if we do not set the keyword to false but give instead a tuple, during the CTMRG routine the algorithm automatically increases the bond dimension
up to a maximal value "χ_max". This happens whenever the truncation error "err_trunc" that we obtain in the creation of the projectors is larger than a choosen tolerance err_tol.
Thus we set the keyword to "adjust_χ = (err_tol, χ_max)" in case we want the bond dimension to be increased during the optimization.

3. You can also return the environment tensors, if you want to use them further. For this just pass the keyword `return_envs = true`. This will return an array of environments, one for every distinct tensor in your unit cell. The environments themselves are `named tuples` and the enviornments are named after their direction (l - left, d- down, r- right, u- up).

For a state, obtained after a few optimization steps of the Heisenberg model we look at the truncation errors as 
a function of the bond dimension - with the corresponding energy. The upper block is for the "fullfishman"-projectors
while the lower block is for the "half"-projectors. We conclude from the convergence of the energy-expectation values
that a maximal trunctation error of ~10^-3 is a reasonable starting point. It can always be chosen more stricly.

```E = -0.6603737287187401 @ χ = 80, maximum(trunc_sv_arr) = 5.975686042268443e-5 
E = -0.6603737358938596 @ χ = 60, maximum(trunc_sv_arr) = 0.00012481759922763688
E = -0.6603737799922393 @ χ = 40, maximum(trunc_sv_arr) = 0.00031194042937930866
E = -0.6603736187795066 @ χ = 30, maximum(trunc_sv_arr) = 0.0006405720791402293
E = -0.6603724597506875 @ χ = 26, maximum(trunc_sv_arr) = 0.0009362616891090525
E = -0.6603639207050201 @ χ = 20, maximum(trunc_sv_arr) = 0.0020002907780533145
E = -0.6603793732337668 @ χ = 16, maximum(trunc_sv_arr) = 0.003927764975811235


E = -0.6603737141782002 @ χ = 80, maximum(trunc_sv_arr) = 8.859010589321488e-5
E = -0.6603736039935315 @ χ = 60, maximum(trunc_sv_arr) = 0.00018689030517925998
E = -0.6603739415374238 @ χ = 40, maximum(trunc_sv_arr) = 0.0004921465970557896
E = -0.6603749442733124 @ χ = 30, maximum(trunc_sv_arr) = 0.0011953646129798726
E = -0.6603675868543281 @ χ = 26, maximum(trunc_sv_arr) = 0.00172234383475717
E = -0.6603589442727148 @ χ = 20, maximum(trunc_sv_arr) = 0.003446092986088426
E = -0.6603566068572576 @ χ = 16, maximum(trunc_sv_arr) = 0.005807880961672133


the function energy_and_gradient is needed for the variational optimization. It calculates the gradient at the fixed point of the CTM-RG iteration.\
It takes the same basic arguments as the function `ctmrg()`.\
It returns the the energy of the state as well as the gradient.

In [7]:
e, gr = energy_and_gradient(loc_in, χ, Pattern_arr; keywords...)

Info: it took 7 CTMRG steps to converge the SV to 1.0e-6
Info: it took 9 CTMRG steps to converge the environments element wise to 1e-6


(0.05835949725883433, TrivialTensorMap{ComplexSpace, 2, 3, Matrix{ComplexF64}}[TensorMap((ℂ^2 ⊗ ℂ^2) ← (ℂ^2 ⊗ ℂ^2 ⊗ (ℂ^8)')):
[:, :, 1, 1, 1] =
  0.005567336781907214 - 0.005928765469616716im  …  -0.011149634650720865 + 0.0008064632806587432im
 -0.009253694487588986 + 0.005699052994547947im      0.000884248082163431 - 0.005658914024379508im

[:, :, 2, 1, 1] =
    0.005345280354359748 + 0.004893615837566665im  …  0.0032025433049552104 + 0.019478296965699804im
 -0.00012230788185149379 - 0.011215039221434029im     -0.003279985624458958 - 0.019177259950587985im

[:, :, 1, 2, 1] =
 -0.004488824711170353 - 0.04542528779528124im   …   -0.01654444849230413 + 0.007929681895767971im
   0.02376771964358964 + 0.002597945232798498im     -0.018211870337401483 + 0.006592175110660186im

[:, :, 2, 2, 1] =
 -0.0022524938588201443 + 0.025286454838490048im  …   0.024012675041133535 + 0.0005035327813669156im
   0.023100150207924986 - 0.017252341543150307im     -0.004726127525362619 - 0.019485953029543493im

Some additional notes (skip on first reading):

1. With the option svd_type = :GKL one can enable the iterative calculation of the singular values during the CTMRG algorithm.
This can be substantially faster for larger bond dimensions.
Note that in some cases, the GKL algorithm, that can be used for the SVD in the calculation of the projectors finds invariant subspaces and fails. 
In these cases the code will fall back to conventional SVD algorithms.

In the "energy_and_gradient" function as well as in the "ctmrg" function we have a keyword "adjust_χ" which is by default set to false. In that case the 
function works normally. However if we do not set the keyword to false but give instead a tuple, during the CTMRG routine the algorithm automatically increases the bond dimension
up to a maximal value "χ_max". This happens whenever the truncation error "err_trunc" that we obtain in the creation of the projectors is larger than a choosen tolerance err_tol.
Thus we set the keyword to "adjust_χ = (err_tol, χ_max)" in case we want the bond dimension to be increased during the optimization.

Now there are two ways to call the "energy_and_gradient" function.\
you can use:\
1. e, gr = energy_and_gradient(loc_in, χ, Pattern_arr; keywords...)\
This version is the standart version. It takes a (local PEPS, environment bond dimension, a pattern array for the unit cell; a bunch of keywords)

2. e, gr = energy_and_gradient(loc_in, χ, Pattern_arr, keywords; adjust_χ = false)\
In this version the only thing to be specified by keyword is "adjust_χ" which is either false of (max_χ, tol)\
TWO IMPORTANT CAVEATS:\
a) in this case χ needs to be an array as [χ]. This is needed for dynamical increase of χ (it needs to be a mutable type e.g. array)\
b) We can still specify the options that were previously specified by keywords. We just pass a tuple (named tuple actually) that contains the keywords we want\
to specify.\
THE SECOND OPTION IS JUST A WRAPPER AROUND THE FIRST VERSION WHICH RETURNS THE ADJUSTED χ VALUE IF IT IS INCREASED. THIS IS NEEDED FOR DYNAMICAL χ INCREASE.\


Now we should the above functions to variationally find the best d=2 approximation to the ground state of the Heisenberg antiferromagnet.

For this we can use gradient based optimizers like L-BFGS. They are implemented in different packages, here we use OptimKit.jl. (an alternative would be Optim.jl)\
for the optimization we need the following two functions:

In [8]:
function _finalize!(x,f,g, numiter)
    
    
    flush(stdout) #this just makes it so the output does not appear all at once.
    flush(stderr)

    
    #here we could save e.g. intermediate output.
    
    return x, f, g
end

function _inner(x, ξ_1, ξ_2)
    return dot(real(ξ_1), real(ξ_2)) + dot(imag(ξ_1), imag(ξ_2))
end

_inner (generic function with 1 method)

we specify the parameters that are used in the optimization. 

In [9]:
optimparas1 = (gradtol = 1e-5, verbosity = 2, maxiter = 100, linesearch = HagerZhangLineSearch(; c₁ = 1e-4, c₂ = 0.9, maxiter = 40))
kwargs = (finalize! = _finalize!, inner = _inner)

(finalize! = _finalize!, inner = _inner)

Here we call the function that optimizes based on our gradient function.\
This function returns the approximation to the ground state `x` found in this procedure, the ground state energy `e` as well as information on the convergence.\
Below we plot some of the results.

In [10]:
x, e, gr, numfg, info = optimize(x -> energy_and_gradient(x, χ, Pattern_arr, keywords; adjust_χ = false), loc_in, LBFGS(15; optimparas1... ); kwargs... )

Info: it took 7 CTMRG steps to converge the SV to 1.0e-6
Info: it took 9 CTMRG steps to converge the environments element wise to 1e-6


┌ Info: LBFGS: initializing with f = 0.058346794754, ‖∇f‖ = 4.3473e-01
└ @ OptimKit /Users/weerde/.julia/packages/OptimKit/xpmbV/src/lbfgs.jl:35


Info: it took 8 CTMRG steps to converge the SV to 1.0e-6
Info: it took 11 CTMRG steps to converge the environments element wise to 1e-6


┌ Info: LBFGS: iter    1: f = -0.554419760558, ‖∇f‖ = 2.8144e-01, α = 1.00e+00, m = 0, nfg = 1
└ @ OptimKit /Users/weerde/.julia/packages/OptimKit/xpmbV/src/lbfgs.jl:76


Info: it took 10 CTMRG steps to converge the SV to 1.0e-6
Info: it took 12 CTMRG steps to converge the environments element wise to 1e-6
Info: it took 9 CTMRG steps to converge the SV to 1.0e-6


...

Info: it took 6 CTMRG steps to converge the SV to 1.0e-6
Info: it took 9 CTMRG steps to converge the environments element wise to 1e-6


┌ Info: LBFGS: iter   46: f = -1.213604608413, ‖∇f‖ = 1.4450e-05, α = 1.00e+00, m = 15, nfg = 1
└ @ OptimKit /Users/weerde/.julia/packages/OptimKit/xpmbV/src/lbfgs.jl:76


Info: it took 6 CTMRG steps to converge the SV to 1.0e-6
Info: it took 9 CTMRG steps to converge the environments element wise to 1e-6


┌ Info: LBFGS: iter   47: f = -1.213604621465, ‖∇f‖ = 2.5421e-05, α = 1.00e+00, m = 15, nfg = 1
└ @ OptimKit /Users/weerde/.julia/packages/OptimKit/xpmbV/src/lbfgs.jl:76
┌ Info: LBFGS: converged after 48 iterations: f = -1.213604634197, ‖∇f‖ = 8.9146e-06
└ @ OptimKit /Users/weerde/.julia/packages/OptimKit/xpmbV/src/lbfgs.jl:138


(TrivialTensorMap{ComplexSpace, 2, 3, Matrix{ComplexF64}}[TensorMap((ℂ^2 ⊗ ℂ^2) ← (ℂ^2 ⊗ ℂ^2 ⊗ (ℂ^8)')):
[:, :, 1, 1, 1] =
 -0.05585303318770048 + 0.13957333869161617im  …  -0.2651692562428464 + 0.46839666257335627im
 -0.11521761747369787 + 0.2955159604795633im       0.3276449887578741 + 0.0024389096932519692im

[:, :, 2, 1, 1] =
 0.0005609171437982964 + 0.06891447530044326im  …  -0.36060080334039785 + 0.5296002231440862im
  -0.17004041416914298 + 0.3910046218562532im       0.40024127983872876 - 0.049570988730022594im

[:, :, 1, 2, 1] =
 0.13086348196974226 + 0.01992839054047904im  …    0.5226392997456976 - 0.1159045377546089im
  0.2701253797040119 - 0.11320695358571657im     -0.22366695905682576 - 0.18198525877824684im

[:, :, 2, 2, 1] =
 0.07439148676875874 + 0.030177120868851416im  …   0.6042221153461318 - 0.1816614075922198im
 0.33670316944167844 - 0.18173397712557002im      -0.3144296420125185 - 0.14850929622017847im

[:, :, 1, 1, 2] =
 -0.10141130410278776 + 0.02212764104420604im

In [11]:
using Plots

In [12]:
pval = plot(1:1:101, info[:,1], color = :blue, legend = false, ylabel = "⟨E⟩", xlabel = "iterations");

pgrad = plot(1:1:101, log10.(info[:,2]), color = :red, ylabel = " ‖∇E‖", legend = false)


In [15]:
pval

In [14]:
pgrad